# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
'''
Gmaps API, heatmap and marker layers based on published solutions to 
6.03 activities '04-Ins_NearestRstr' and '10-Stu_BankDeserts_Heatmap'
'''

# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json
from config import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#load *.csv from previous section
filepath = "weatherpy.csv" 
weather_data = pd.read_csv(filepath)
weather_data.head()


,Unnamed: 0,City,Country,Lat,Lng,Temp,Humidity,Wind Speed,Cloudiness
0,0,hobart,AU,-42.88,147.33,50.95,66,1.12,75
1,1,babusnica,RS,43.06,22.41,55.99,10,4.99,100
2,2,tiksi,RU,71.69,128.87,3.02,93,4.68,96
3,3,malanje,AO,-9.54,16.34,67.15,87,1.66,99
4,4,altay,CN,47.87,88.12,25.03,81,1.74,0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Cleaning blanks with .dropna()
weather_cleaned = weather_data.dropna()

# Store 'Lat' and 'Lng' into  locations, ensure datatype is float
locations = weather_cleaned[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humidity = weather_cleaned["Humidity"].astype(float)

#Declare fig to simplify calling Google maps
fig = gmaps.figure()

#Create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

#Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#narrow down weather criteria
'''
  * A max temperature lower than 80 degrees but higher than 70.
  * Wind speed less than 10 mph.
  * Zero cloudiness.
  * Drop any rows that don't contain all three conditions.
'''

best_locations = weather_cleaned.loc[(weather_cleaned["Temp"]>= 70)  & 
                                     (weather_cleaned["Temp"]<= 80)  &
                                     (weather_cleaned["Wind Speed"]<=5) &
                                     (weather_cleaned["Cloudiness"]==0)] 

#Print data - no null values
best_locations


,Unnamed: 0,City,Country,Lat,Lng,Temp,Humidity,Wind Speed,Cloudiness
209,209,umm lajj,SA,25.02,37.27,78.78,72,3.56,0
278,278,malkapur,IN,20.88,76.20,73.33,49,2.46,0
387,387,salalah,OM,17.02,54.09,77.00,33,3.36,0
429,429,rafaela,AR,-31.25,-61.49,77.83,16,3.00,0
468,468,matehuala,MX,23.65,-100.65,78.06,18,2.91,0
518,518,lodja,CD,-3.48,23.43,70.75,90,2.33,0


In [9]:
#Ensure latitude, longitude and humidity are floats
locations = best_locations[["Lat", "Lng"]].astype(float)
humidity = best_locations["Humidity"].astype(float)

'''
# Create a  seatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
'''

#Add layer to map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
#Creating hotel_df
hotel_df = best_locations.loc[:,["City","Country","Lat","Lng"]]

#Add blank "Hotel Name" column as string
hotel_df["Hotel Name"]=""

hotel_df

,City,Country,Lat,Lng,Hotel Name
66,altamira,BR,-3.20,-52.21,
108,saint-philippe,RE,-21.36,55.77,
182,bonthe,SL,7.53,-12.51,
287,manacapuru,BR,-3.30,-60.62,
310,odweyne,SO,9.41,45.06,
444,ormara,PK,25.21,64.64,


In [76]:
#Setup Google maps nearby places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Build search parameters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#Iterate through rows in df
for index, row in hotel_df.iterrows():
    #Store lat, lng and city name
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #Format location string, add to parameters parameters
    params["location"] = f"{lat},{lng}"

    #Assemble url and make API request, store response
    print(f"API request for {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #Extract results
    results = response['results']
    
    #Print results and save the hotel name to dataframe
    try:
        print(f"Hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    #If no hotel is returned, print 'not found'
    except (KeyError, IndexError):
        print(f"No hotel in 5km of {city_name} has been found.")
        
  

Retrieving Results for Index 66: altamira.
Closest hotel in altamira is Hotel Xingu.
------------
Retrieving Results for Index 108: saint-philippe.
Closest hotel in saint-philippe is Les Embruns Du Baril.
------------
Retrieving Results for Index 182: bonthe.
Closest hotel in bonthe is Bonthe Holiday Village.
------------
Retrieving Results for Index 287: manacapuru.
Closest hotel in manacapuru is Hotel Amazon Experience.
------------
Retrieving Results for Index 310: odweyne.
Closest hotel in odweyne is Al shukri hotel.
------------
Retrieving Results for Index 444: ormara.
Closest hotel in ormara is Alqadri hotel zero point ormara.
------------


In [77]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
66,altamira,BR,-3.20,-52.21,Hotel Xingu
108,saint-philippe,RE,-21.36,55.77,Les Embruns Du Baril
182,bonthe,SL,7.53,-12.51,Bonthe Holiday Village
287,manacapuru,BR,-3.30,-60.62,Hotel Amazon Experience
310,odweyne,SO,9.41,45.06,Al shukri hotel
444,ormara,PK,25.21,64.64,Alqadri hotel zero point ormara


In [79]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = 
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [87]:
# Convert hotel name column to list
hotel_list = hotel_df["Hotel Name"].tolist()

# Create symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content=[f"Hotel Name: {hotel_list}" for hotel in hotel_list])

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates) --- #No such variable?

# Add the layer to the map
fig.add_layer(markers)

#fig = gmaps.figure() --think this has already been declared

'''
locations = best_locations[["Lat", "Lng"]].astype(float)
humidity = best_locations["Humidity"].astype(float)

#Ensure latitude, longitude and humidity are floats
locations = best_locations[["Lat", "Lng"]].astype(float)
humidity = best_locations["Humidity"].astype(float)
'''
#Add  layers
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

#Print map
fig



Figure(layout=FigureLayout(height='420px'))